In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
#set path for my_tools script
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from cmap import ncl_colormap
from my_tools import file_dic, plot_dic
import matplotlib.ticker as plticker
from mpl_toolkits.basemap import Basemap, addcyclic
import string
from matplotlib.patches import Polygon
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings("ignore")
import matplotlib



In [2]:
path = '/media/onno/Volume/'

In [3]:
xx, yy = np.meshgrid(np.linspace(-1, 1, 100), np.linspace(-1, 1, 100))
zz = xx**2 + yy**2

my_cmap_rgb = plt.get_cmap('bwr')(np.arange(256))
alpha = 0.65

for i in range(3): # Do not include the last column!
    my_cmap_rgb[:,i] = (1 - alpha) + alpha*my_cmap_rgb[:,i]
my_cmap = matplotlib.colors.ListedColormap(my_cmap_rgb, name='my_cmap')

In [4]:
file_env = 'era51_mars_env_wledit2000-10000_latavg_v300_79-19_6hourly_smoothed.nc'
file_cp = 'era51_mars_phasevel_wledit2000-10000_latavg_v300_envgt15_79-19_6hourly_setvrange_-100to100.nc'
file_t850 = 'era51_mars_t850_79-19_6hourly_anom_from_smoothed04_clim.nc'
file_v300 = 'era51_mars_v300_wledit2000-10000_latavg_79-19_6hourly.nc'
file_z500 = 'era51_mars_phi500_79-19_6hourly.nc'
file_mslp = 'era5_mslp_79-19_6hourly_remapbil2x2.nc'
file_prec = 'era5_totprecip_79-19_1hourly_remapbil2x2_daysum.nc'
file_z500_anom = 'era51_mars_phi500_79-19_6hourly_anom_from_smoothed04_clim_daymean.nc'
file_z500_fcst = 'era5rf_z500_0-240h_12hourly_2x2nh_jan79-dec19.nc'


# era_env = xr.open_dataset(path+file_env)
# era_cp = xr.open_dataset(path+file_cp)
# era_v300 = xr.open_dataset(path+file_v300)
era_z500 = xr.open_dataset(path+file_z500)
era_mslp = xr.open_dataset(path+file_mslp)
# era_t850 = xr.open_dataset(path+file_t850)
# era_prec = xr.open_dataset(path+file_prec)
era_z500_anom = xr.open_dataset(path+file_z500_anom)
era5rf_z500 = xr.open_dataset(path+file_z500_fcst,decode_times=False)
# era_prec['time'] = [era_prec['time'].values[i] - pd.Timedelta(15,'h') if i==0 else 
#                    era_prec['time'].values[i] - pd.Timedelta(11.5,'h')
#                     for i in range(len( era_prec['time'].values))]
# era_prec = era_prec.drop('time_bnds')
era_z500_anom['time'] = [era_z500_anom['time'].values[i] - pd.Timedelta(9,'h')
                    for i in range(len(era_z500_anom['time'].values))]
era_z500_anom = era_z500_anom.drop('time_bnds')
ERA5RF_init_time = pd.Timestamp('1900-01-01')
era5rf_z500['time']=[pd.Timedelta(i,'hours')+ERA5RF_init_time for i in era5rf_z500.time.values]


composite MSLP and Z500

In [7]:
plt.rcParams['hatch.color'] = 'w'
seasonz = ['DJF','MAM','JJA','SON']
modelz = ['ERA5RF']
eventz = ['persistent_hw','persistent_cw']
event_labelz = ['Warm Extreme','Cold Extreme']
regionz = ['Finland']
file_warm_mslp = '/fcst_composite/reanalysis/composite_mslp_warm_forecasts_{}_{}_{}.nc'
file_cold_mslp = '/fcst_composite/reanalysis/composite_mslp_cold_forecasts_{}_{}_{}.nc'
# file_p_mslp = '/fcst_composite/reanalysis/composite_mslp_diff_p_value_{}_{}_{}_{}.nc'
file_warm_z500 = '/fcst_composite/reanalysis/composite_z500_warm_forecasts_{}_{}_{}.nc'
file_cold_z500 = '/fcst_composite/reanalysis/composite_z500_cold_forecasts_{}_{}_{}.nc'
# file_p_z500 = '/fcst_composite/reanalysis/composite_z500_diff_p_value_{}_{}_{}_{}.nc'
file_diff_warm_z500 = '/fcst_composite/reanalysis/diff_z500_warm_forecasts_{}_{}_{}.nc'
file_diff_cold_z500 = '/fcst_composite/reanalysis/diff_z500_cold_forecasts_{}_{}_{}.nc'

for region in regionz:
    for model in modelz:
        for season in seasonz:
            fig,axz = plt.subplots(5,2,figsize=(18,10))
            ds_warm_z500 = xr.open_dataset(path+file_warm_z500.format(season,region,model))
            ds_cold_z500 = xr.open_dataset(path+file_cold_z500.format(season,region,model))     
            ds_diff_warm_z500 = xr.open_dataset(path+file_diff_warm_z500.format(season,region,model)).sel(latitude=slice(85,0))
            ds_diff_cold_z500 = xr.open_dataset(path+file_diff_cold_z500.format(season,region,model)).sel(latitude=slice(85,0))               
            ds_warm_mslp = xr.open_dataset(path+file_warm_mslp.format(season,region,model))
            ds_cold_mslp = xr.open_dataset(path+file_cold_mslp.format(season,region,model))      
            days_to_plot = np.arange(-4,5,2)
            for i in range(len(days_to_plot)):
                ds_warm_z500_day = ds_warm_z500.sel(time=days_to_plot[i])
                ds_warm_z500_day_cyclic, lon_cyclic = addcyclic(ds_warm_z500_day.z.values, ds_warm_z500_day.longitude.values)
                ds_diff_warm_z500_day = ds_diff_warm_z500.sel(time=days_to_plot[i])
                ds_diff_warm_z500_day_cyclic, lon_cyclic_diff = addcyclic(ds_diff_warm_z500_day.z.values, ds_diff_warm_z500_day.longitude.values)                

                ds_warm_mslp_day = ds_warm_mslp.sel(time=days_to_plot[i])
                ds_warm_mslp_day_cyclic, lon_cyclic = addcyclic(ds_warm_mslp_day.var151.values, ds_warm_mslp_day.longitude.values)

                lon,lat = np.meshgrid(lon_cyclic,ds_warm_z500_day.latitude.values)
                lon_diff,lat_diff = np.meshgrid(lon_cyclic_diff,ds_diff_warm_z500_day.latitude.values)

                ax_warm = axz[i,0]
                ax_warm.set_title('Warm Extremes Day {}'.format(days_to_plot[i]))
                m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_warm,
                            llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                m.drawcoastlines()
                m.contourf(lon,lat,ds_warm_z500_day_cyclic/(9.80665*10),latlon='true',
                          vmin=450,vmax=600,cmap=ncl_colormap(),levels=np.linspace(450,600,30),extend='both')
                m.contour(lon,lat,ds_warm_mslp_day_cyclic/100,latlon='true',
                          vmin=970,vmax=1040,levels=np.arange(970,1041,5),cmap='Greys',linewidths=.75)
                m.contour(lon_diff,lat_diff,ds_diff_warm_z500_day_cyclic,latlon='true',
                          vmin=-6,vmax=6,levels=[-6,-4,-2,2,4,6],cmap='bwr',extend='both')

                ds_cold_z500_day = ds_cold_z500.sel(time=days_to_plot[i])
                ds_cold_z500_day_cyclic, lon_cyclic = addcyclic(ds_cold_z500_day.z.values, ds_cold_z500_day.longitude.values)
                ds_diff_cold_z500_day = ds_diff_cold_z500.sel(time=days_to_plot[i])
                ds_diff_cold_z500_day_cyclic, lon_cyclic = addcyclic(ds_diff_cold_z500_day.z.values, ds_diff_cold_z500_day.longitude.values)                

                ds_cold_mslp_day = ds_cold_mslp.sel(time=days_to_plot[i])
                ds_cold_mslp_day_cyclic, lon_cyclic = addcyclic(ds_cold_mslp_day.var151.values, ds_cold_mslp_day.longitude.values)

                lon,lat = np.meshgrid(lon_cyclic,ds_cold_z500_day.latitude.values)
                lon_diff,lat_diff = np.meshgrid(lon_cyclic_diff,ds_diff_cold_z500_day.latitude.values)

                ax_cold = axz[i,1]
                ax_cold.set_title('Cold Extremes Day {}'.format(days_to_plot[i]))

                m = Basemap(projection='cyl',lat_0=50,lon_0=0,resolution='c',ax=ax_cold,
                            llcrnrlon=-180,llcrnrlat=15,urcrnrlon=180,urcrnrlat=75)
                m.drawcoastlines()
                im = m.contourf(lon,lat,ds_cold_z500_day_cyclic/(9.80665*10),latlon='true',
                          vmin=450,vmax=600,cmap=ncl_colormap(),levels=np.linspace(450,600,30),extend='both')
                m.contour(lon,lat,ds_cold_mslp_day_cyclic/100,latlon='true',
                          vmin=970,vmax=1040,levels=np.arange(970,1041,5),cmap='Greys',linewidths=.75)
                m.contour(lon_diff,lat_diff,ds_diff_cold_z500_day_cyclic,latlon='true',
                          vmin=-6,vmax=6,levels=[-6,-4,-2,2,4,6],cmap='bwr',extend='both')



            fig.subplots_adjust(left=0.025,right=0.975,top=0.95,bottom=0.105,wspace=0,hspace=0.1)     
            fig.suptitle('Composites Z500 (contours), Z500 forecast error and MSLP (contour lines) {} {} {}'.format(season,region,model))

            cbax = fig.add_axes([0.2,0.07,0.6,0.015])

            cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.arange(450,601,25))
            cbar.ax.set_xlabel('Z 500 hPa [m]')
            sys.exit()
            fig.savefig(path+'fcst_composite/reanalysis/figures/composite_z500_mslp_{}_{}_{}_error.png'.format(season,region,model))
#                 fig.savefig(path+'fcst_composite/reanalysis/figures/try.png'.format(event,season,region,model))
            plt.close(fig)


SystemExit: 

composite MSLP and Z500 anomaly

In [9]:
plt.rcParams['hatch.color'] = 'k'
seasonz = ['DJF','JJA']
modelz = ['ERA5RF']
eventz = ['persistent_hw','persistent_cw']
regionz = ['Finland']
event_labelz = ['Warm Extreme','Cold Extreme']
file_warm_z500 = '/fcst_composite/reanalysis/composite_z500_warm_forecasts_{}_{}_{}.nc'
file_cold_z500 = '/fcst_composite/reanalysis/composite_z500_cold_forecasts_{}_{}_{}.nc'
file_warm_z500_anom = '/fcst_composite/reanalysis/composite_z500_anom_warm_forecasts_{}_{}_{}.nc'
file_cold_z500_anom = '/fcst_composite/reanalysis/composite_z500_anom_cold_forecasts_{}_{}_{}.nc'
file_diff_warm_z500 = '/fcst_composite/reanalysis/diff_z500_warm_forecasts_{}_{}_{}.nc'
file_diff_cold_z500 = '/fcst_composite/reanalysis/diff_z500_cold_forecasts_{}_{}_{}.nc'

for region in regionz:
    for model in modelz:
        fig,axz = plt.subplots(5,4,figsize=(10,10))
        for j,season in enumerate(seasonz):
            ds_warm_z500 = xr.open_dataset(path+file_warm_z500.format(season,region,model))
            ds_cold_z500 = xr.open_dataset(path+file_cold_z500.format(season,region,model))
            ds_diff_warm_z500 = xr.open_dataset(path+file_diff_warm_z500.format(season,region,model)).sel(latitude=slice(85,0))
            ds_diff_cold_z500 = xr.open_dataset(path+file_diff_cold_z500.format(season,region,model)).sel(latitude=slice(85,0))  
            ds_warm_z500_anom = xr.open_dataset(path+file_warm_z500_anom.format(season,region,model))
            ds_cold_z500_anom = xr.open_dataset(path+file_cold_z500_anom.format(season,region,model))      
            days_to_plot = np.arange(-4,5,2)
            for i in range(len(days_to_plot)):
                ds_warm_z500_day = ds_warm_z500.sel(time=days_to_plot[i])
                ds_warm_z500_day_cyclic, lon_cyclic = addcyclic(ds_warm_z500_day.z.values, ds_warm_z500_day.longitude.values)
                ds_diff_warm_z500_day = ds_diff_warm_z500.sel(time=days_to_plot[i])
                ds_diff_warm_z500_day_cyclic, lon_cyclic_diff = addcyclic(ds_diff_warm_z500_day.z.values, ds_diff_warm_z500_day.longitude.values)                
                ds_warm_z500_anom_day = ds_warm_z500_anom.sel(time=days_to_plot[i])
                ds_warm_z500_anom_day_cyclic, lon_cyclic = addcyclic(ds_warm_z500_anom_day.z.values, ds_warm_z500_anom_day.longitude.values)                
                lon,lat = np.meshgrid(lon_cyclic,ds_warm_z500_day.latitude.values)
                lon_diff,lat_diff = np.meshgrid(lon_cyclic_diff,ds_diff_warm_z500_day.latitude.values)


                ax_warm = axz[i,j*2]
                ax_warm.set_title('Warm {} Day {}'.format(season,days_to_plot[i]),pad=4)
                m = Basemap(projection='merc',lat_0=50,lon_0=0,resolution='c',ax=ax_warm,
                            llcrnrlon=-60,llcrnrlat=20,urcrnrlon=60,urcrnrlat=70)
                m.drawcoastlines()
                m.contour(lon,lat,ds_warm_z500_day_cyclic/(9.80665*10),latlon='true',colors='k',
                          vmin=450,vmax=600,levels=np.arange(450,601,10),linewidths=.5)
                m.contourf(lon,lat,ds_warm_z500_anom_day_cyclic/(9.80665*10),latlon='true',
                          vmin=-25,vmax=25,cmap=my_cmap,levels=np.linspace(-25,25,501),extend='both')
                m.contour(lon_diff,lat_diff,ds_diff_warm_z500_day_cyclic,latlon='true',
                          vmin=-6,vmax=6,levels=[-6,-4,-2,2,4,6],cmap='seismic',extend='both')

                ds_cold_z500_day = ds_cold_z500.sel(time=days_to_plot[i])
                ds_cold_z500_day_cyclic, lon_cyclic = addcyclic(ds_cold_z500_day.z.values, ds_cold_z500_day.longitude.values)
                ds_diff_cold_z500_day = ds_diff_cold_z500.sel(time=days_to_plot[i])
                ds_diff_cold_z500_day_cyclic, lon_cyclic_diff = addcyclic(ds_diff_cold_z500_day.z.values, ds_diff_cold_z500_day.longitude.values)                
                ds_cold_z500_anom_day = ds_cold_z500_anom.sel(time=days_to_plot[i])
                ds_cold_z500_anom_day_cyclic, lon_cyclic = addcyclic(ds_cold_z500_anom_day.z.values, ds_cold_z500_anom_day.longitude.values)
                lon,lat = np.meshgrid(lon_cyclic,ds_cold_z500_day.latitude.values)
                lon_diff,lat_diff = np.meshgrid(lon_cyclic_diff,ds_diff_cold_z500_day.latitude.values)


                ax_cold = axz[i,j*2+1]
                ax_cold.set_title('Cold {} Day {}'.format(season,days_to_plot[i]),pad=4)

                m = Basemap(projection='merc',lat_0=50,lon_0=0,resolution='c',ax=ax_cold,
                            llcrnrlon=-60,llcrnrlat=20,urcrnrlon=60,urcrnrlat=70)
                m.drawcoastlines()
                m.contour(lon,lat,ds_cold_z500_day_cyclic/(9.80665*10),latlon='true',colors='k',
                          vmin=450,vmax=600,levels=np.arange(-25,601,10),linewidths=.5)
                im = m.contourf(lon,lat,ds_cold_z500_anom_day_cyclic/(9.80665*10),latlon='true',
                          vmin=-25,vmax=25,cmap=my_cmap,levels=np.linspace(-25,25,501),extend='both')
                m.contour(lon_diff,lat_diff,ds_diff_cold_z500_day_cyclic,latlon='true',
                          vmin=-6,vmax=6,levels=[-6,-4,-2,2,4,6],cmap='seismic',extend='both')



        fig.subplots_adjust(left=0.025,right=0.975,top=0.94,bottom=0.07,wspace=0,hspace=0.125)     
        fig.suptitle('Z500 Anomaly (contours) and Z500 (contour lines) {} {} {}'.format(season,region,model))

        cbax = fig.add_axes([0.2,0.05,0.6,0.015])

        cbar = fig.colorbar(im,cax=cbax,orientation='horizontal',ticks=np.linspace(-25,25,11))
        cbar.ax.set_xlabel('Z\' 500 hPa [m]')

        fig.savefig(path+'/Main_Figures/composite_z500_anom_z500_error_alpha_Finland.png')
        sys.exit()
        plt.close(fig)



SystemExit: 

In [8]:
ds_diff_warm_z500_day_cyclic.max()

7.6919804